In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34305")
client

<Client: 'tcp://127.0.0.1:34305' processes=8 threads=40, memory=157.36 GiB>

In [3]:
import numpy as np
import xarray as xr
import pandas as pd
import os.path as op
import os
import gsw
from xgcm.grid import Grid
from xmitgcm import open_mdsdataset
from fastjmd95 import rho as densjmd95
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
ddir = '/tank/chaocean/qjamet/RUNS/ORAR/reruns2/'
gdir = '/tank/chaocean/grid_chaO/gridMIT_update1/'
chaos = '/tank/chaocean/tuchida/'
# savedir = '/tank/chaocean/tuchida/ECycle/'
savedir = '/tank/spectre/tuchida/chaocean/Snap/'

In [5]:
grav = 9.81
nensembs = 5
Kh = 20.
K4 = 1e10
Kr = 1e-5
thetaMax = 1e20
rhoConst = 9.998e2
# Cp = 3.994e3
Cp = 3989.244953
# years = np.arange(2008,2013,dtype=int)
ySstart = -15
yNstart = 15
yend = 50
dnf = 2160
nfile = 73

xchunk = 100
ychunk = 100

nmembs = np.arange(100,148)

In [6]:
years = np.arange(1967,1968,dtype=int)
ntimes = np.arange(1419120,1576800,dnf).reshape(len(years),nfile)
dd = 0  # 
yystart = 0
ntimes[yystart,dd:]

array([1419120, 1421280, 1423440, 1425600, 1427760, 1429920, 1432080,
       1434240, 1436400, 1438560, 1440720, 1442880, 1445040, 1447200,
       1449360, 1451520, 1453680, 1455840, 1458000, 1460160, 1462320,
       1464480, 1466640, 1468800, 1470960, 1473120, 1475280, 1477440,
       1479600, 1481760, 1483920, 1486080, 1488240, 1490400, 1492560,
       1494720, 1496880, 1499040, 1501200, 1503360, 1505520, 1507680,
       1509840, 1512000, 1514160, 1516320, 1518480, 1520640, 1522800,
       1524960, 1527120, 1529280, 1531440, 1533600, 1535760, 1537920,
       1540080, 1542240, 1544400, 1546560, 1548720, 1550880, 1553040,
       1555200, 1557360, 1559520, 1561680, 1563840, 1566000, 1568160,
       1570320, 1572480, 1574640])

In [7]:
dsg = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                              % (100)), 
                      grid_dir=gdir,
                      prefix=['snap_diagOcn'], delta_t=2e2,
                      iters=ntimes[0,0], 
                     ).chunk({'YC':ychunk,'YG':ychunk,
                              'XC':xchunk,'XG':xchunk})
grid = Grid(dsg, periodic=['X'])
Vol = (dsg.rA * dsg.drF * dsg.hFacC).where(dsg.maskC!=0.)
dsg

<xarray.Dataset>
Dimensions:  (time: 1, XC: 1000, YC: 900, XG: 1000, YG: 900, Z: 46, Zp1: 47,
              Zu: 46, Zl: 46)
Coordinates: (12/33)
    iter     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) timedelta64[ns] 3285 days
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * XG       (XG) >f4 262.0 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.8 347.9
  * YG       (YG) >f4 -19.96 -19.87 -19.79 -19.71 ... 54.71 54.79 54.88 54.96
    ...       ...
    maskW    (Z, YC, XG) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    rhoRef   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    maskInS  (YG, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
Data variables:
    THETA    (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    SALT     (time, Z, YC, XC) >f4 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    UVEL     (time, Z, YC, XG) >f4 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    VVEL     (time, Z, YG, XC) >f4 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
    WVEL     (time, Zl, YC, XC) >f4 dask.array<chunksize=(1, 46, 100, 100), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Created by calling `open_mdsdataset(grid_dir='/tank/chaocea...

In [7]:
nmemb = 100
for itime in ntimes[yystart,:]:
    
    if itime == ntimes[yystart,0]:
        ds = open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                          % (nmemb)), 
                                  grid_dir=gdir,
                                  prefix=['snap_diagOcn'], delta_t=2e2,
                                  iters=itime, 
                                 ).chunk({'YC':ychunk,'YG':ychunk,
                                                'XC':xchunk,'XG':xchunk})
    else:
        ds = xr.concat([ds, open_mdsdataset(op.join(ddir,'memb%03d/run1967/ocn/' 
                                                          % (nmemb)), 
                                                  grid_dir=gdir,
                                                  prefix=['snap_diagOcn'], delta_t=2e2,
                                                  iters=itime, 
                                                 ).chunk({'YC':ychunk,'YG':ychunk,
                                                                'XC':xchunk,'XG':xchunk})
                           ], dim='time')
print(ds)

dsm = ds.reset_coords(drop=True).mean('time')
grid = Grid(dsm, periodic=['X'])
    
wp = ds.WVEL - dsm.WVEL
vp = ds.VVEL - dsm.VVEL
up = ds.UVEL - dsm.UVEL
    
uu = (up**2).mean('time')
vu = (grid.interp(grid.interp(vp,'X'),'Y',boundary='extend')
          * up).mean('time')
wu = (grid.interp(grid.interp(wp,'X'),'Z',boundary='extend')
          * up).mean('time')
    
uv = (grid.interp(grid.interp(up,'X'),'Y',boundary='extend')
          * vp).mean('time')
vv = (vp**2).mean('time')
wv = (grid.interp(grid.interp(wp,'Y',boundary='extend'),'Z',boundary='extend')
          * vp).mean('time')
    
    
xAW = (ds.dyG * ds.drF * ds.hFacW).reset_coords(drop=True)
xAS = (ds.dyC * ds.drF * ds.hFacS).reset_coords(drop=True)
yAW = (ds.dxC * ds.drF * ds.hFacW).reset_coords(drop=True)
yAS = (ds.dxG * ds.drF * ds.hFacS).reset_coords(drop=True)
hFacF = grid.interp(dsg.hFacS,"X")
    
ux = (grid.diff(dsm.UVEL*xAW,'X') 
          / dsg.drF/dsg.rA/dsg.hFacC)
uy = (grid.diff(dsm.UVEL*yAW,'Y',boundary='extend') 
          / dsg.drF/dsg.rAz/hFacF)
uz = (-grid.diff(dsm.UVEL,'Z',boundary='extend') 
          / xr.DataArray(dsg.drC[:-1].data,dims='Zl'))
    
vx = (grid.diff(dsm.VVEL*xAS,'X') 
          / dsg.drF/dsg.rAz/hFacF)
vy = (grid.diff(dsm.VVEL*yAS,'Y',boundary='extend') 
          / dsg.drF/dsg.rA/dsg.hFacC)
vz = (-grid.diff(dsm.VVEL,'Z',boundary='extend') 
          / xr.DataArray(dsg.drC[:-1].data,dims='Zl'))
    
Pr = ((grid.interp(uu,'X')*ux
           + grid.interp(vu*grid.interp(uy,'Y',boundary='extend'),'X')
           + grid.interp(wu*grid.interp(uz,'Z',boundary='extend'),'X')
          )
          + (grid.interp(uv*grid.interp(vx,'X'),'Y',boundary='extend')
             + grid.interp(vv,'Y',boundary='extend')*vy
             + grid.interp(wv*grid.interp(vz,'Z',boundary='extend'),'Y',boundary='extend')
            )
         )
            
dsave = Pr.reset_coords(drop=True).chunk({'Z':4,'YC':ychunk,'XC':xchunk}
                                            ).to_dataset(name='ShProd')
dsave.to_zarr(op.join(savedir,'%4d/Time-decomp/S-P.zarr' 
                          % (years[yystart])), 
                  mode='w')
print(dsave)
dsave.close()
ds.close()
dsm.close()
del Pr

<xarray.Dataset>
Dimensions:  (time: 73, XC: 1000, YC: 900, XG: 1000, YG: 900, Z: 46, Zp1: 47,
              Zu: 46, Zl: 46)
Coordinates: (12/33)
    iter     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) timedelta64[ns] 3285 days 3290 days ... 3640 days 3645 days
  * XC       (XC) >f4 262.0 262.1 262.2 262.3 262.3 ... 347.7 347.8 347.9 348.0
  * YC       (YC) >f4 -19.92 -19.83 -19.75 -19.67 ... 54.75 54.83 54.92 55.0
  * XG       (XG) >f4 262.0 262.0 262.1 262.2 262.3 ... 347.7 347.8 347.8 347.9
  * YG       (YG) >f4 -19.96 -19.87 -19.79 -19.71 ... 54.71 54.79 54.88 54.96
    ...       ...
    maskW    (Z, YC, XG) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskS    (Z, YG, XC) bool dask.array<chunksize=(46, 100, 100), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(100, 100), meta=np.ndarray>
    rhoRef   (Z) >f4 dask.array<chunksize=(46,), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(100, 100), 

In [8]:
SP = xr.open_zarr(op.join(savedir,'%4d/Time-decomp/S-P.zarr' 
                                  % (years[yystart]))
                         ).ShProd

ymax = 43
ymin = 14
xmin = 270
xmax = 337
  
((SP*Vol).sel(
           Z=slice(None,-1e3),YC=slice(ymin,ymax),
           XC=slice(xmin,xmax)
          ).sum(['Z','YC','XC'],skipna=True)
 / Vol.sel(
           Z=slice(None,-1e3),YC=slice(ymin,ymax),
           XC=slice(xmin,xmax)
          ).sum(['Z','YC','XC'],skipna=True)
).reset_coords(drop=True).compute()

<xarray.DataArray ()>
array(-1.4521826e-09, dtype=float32)